In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

#Tipos de capas en torch

In [ ]:
#Vemos cómo es una capa Softmax: https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html
m = nn.Softmax(dim=1)
input = torch.randn(1, 3)
print("La entrada es: \n{}".format(input))
output = m(input)
print("La salida es: \n{}".format(output))
print("La suma de la salida es\n{}".format(output.sum()))

In [ ]:
#Vemos cómo es una capa Sigmoide: https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html
m = nn.Sigmoid()
input = torch.randn(1, 3)
print("La entrada es: \n{}".format(input))
output = m(input)
print("La salida es: \n{}".format(output))
print("La suma de la salida es\n{}".format(output.sum()))

In [ ]:
#Vemos cómo es una capa Conv2D: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
m = nn.Conv2d(1, 2, 2)
#los argumentos son: 1- canales de entrada, 2- canales de salida, 3- medida del kernel
#tiene 2 filtros
input = torch.randn(1,32,32)
print("La size de entrada es: \n{}".format(input.shape))
output = m(input)
print("La size de salida es: \n{}".format(output.shape))

In [ ]:
#Capa de Pooling: https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html
m = nn.MaxPool2d(2)
input = torch.randn(1,4,4)
print("La entrada es: \n{}".format(input))

output = m(input)
print("La salida es: \n{}".format(output))

In [ ]:
#Capa lineal (fully connected): https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
m = nn.Linear(20, 128)
input = torch.randn(128,20)
print("La size de entrada es: \n{}".format(input.shape))
output = m(input)
print("La size de salida es: \n{}".format(output.shape))

In [ ]:
#Capa dropout: https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
m = nn.Dropout(p=0.5)
input = torch.randn(3, 3)
print("La entrada es: \n{}".format(input))
output = m(input)
print("La salida es: \n{}".format(output))

#Ejemplo de CNN

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        #definiremos en init todo lo que tenga parámetros entrenables.
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.dropout1 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #NO hace falta poner en el init lo que no tenga parámetros entrenables (ejemplo: relu)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        # Pass data through dropout1
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



net = Net()
net.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
net.train()

for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
outputs = net(images.to(device))

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')